In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd

In [ ]:
# Load the dataset
df = pd.read_csv("US_Election_Counties_Dataset.csv")

# Define columns to drop
columns_to_drop = [
    'Unnamed: 0', 'county', 'state',
    '2020 Democrat vote raw', '2020 Democrat vote %',
    '2020 Republican vote raw', '2020 Republican vote %',
    '2020 other vote raw', '2020 other vote %',
    'Voted Democrat'
]

# Select features and target and convert to NumPy array
X = df.drop(columns=columns_to_drop).values
y = df['Voted Democrat'].values

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a Normalization layer
normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(X_train)

# Normalize the features
X_train_normalized = normalizer(X_train)
X_test_normalized = normalizer(X_test)

In [ ]:
# Build the FNN model
def create_fnn_model(input_dim):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Input(shape=(input_dim,)),
        tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(32, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss='binary_crossentropy',
                  metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
    return model

# Create the model
input_dim = X_train_normalized.shape[1]
model = create_fnn_model(input_dim)

In [ ]:
# Add Early Stopping
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

# Train the model
history = model.fit(
    X_train_normalized, y_train,
    validation_data=(X_test_normalized, y_test),
    epochs=50,
    batch_size=32,
    callbacks=[early_stopping]
)

Epoch 1/50
79/79 ━━━━━━━━━━━━━━━━━━━━ 7s 12ms/step - accuracy: 0.8281 - loss: 0.5849 - precision_1: 0.6507 - recall_1: 0.2116 - val_accuracy: 0.8855 - val_loss: 0.3660 - val_precision_1: 0.7662 - val_recall_1: 0.5221
Epoch 2/50
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8987 - loss: 0.3316 - precision_1: 0.7639 - recall_1: 0.5959 - val_accuracy: 0.8887 - val_loss: 0.3324 - val_precision_1: 0.7216 - val_recall_1: 0.6195
Epoch 3/50
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9194 - loss: 0.2783 - precision_1: 0.8092 - recall_1: 0.7075 - val_accuracy: 0.8919 - val_loss: 0.3201 - val_precision_1: 0.7228 - val_recall_1: 0.6460
Epoch 4/50
79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9237 - loss: 0.2690 - precision_1: 0.8368 - recall_1: 0.7153 - val_accuracy: 0.9030 - val_loss: 0.3070 - val_precision_1: 0.7708 - val_recall_1: 0.6549
Epoch 5/50
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9325 - loss: 0.2483 - precision_1: 0.8592 - recall_1: 0.7555 - val_accu

In [ ]:
# Evaluate the model
test_loss, test_accuracy, test_precision, test_recall = model.evaluate(X_test_normalized, y_test, verbose=0)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Test Precision: {test_precision:.4f}")
print(f"Test Recall: {test_recall:.4f}")

# Compute F1 score
predictions = model.predict(X_test_normalized)
predictions_binary = (predictions > 0.5).astype(int)
f1 = f1_score(y_test, predictions_binary)
print(f"F1 Score: {f1:.4f}")

Test Loss: 0.2489
Test Accuracy: 0.9173
Test Precision: 0.7748
Test Recall: 0.7611
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
F1 Score: 0.7679


In [ ]:
# Save the trained model
model.save('fnn_voting_model.keras')

In [ ]:
# Testing the saved model on the states dataset

# Load the states dataset
states_df = pd.read_csv("US_Election_States_Dataset.csv")

# Define columns to drop (based on the training dataset preprocessing)
columns_to_drop = [
    'Unnamed: 0', 'state', '2020 Democrat vote raw', '2020 Democrat vote %',
    '2020 Republican vote raw', '2020 Republican vote %',
    '2020 other vote raw', '2020 other vote %',
    'Voted Democrat', 'State Abbreviations'
]

# Select features and target and convert to NumPy array
X_states = states_df.drop(columns=columns_to_drop).values
y_states = states_df['Voted Democrat'].values

# Normalize the features using the normalization layer
normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(X_states)
X_states_normalized = normalizer(X_states)

# Load the saved model
fnn_model = tf.keras.models.load_model('fnn_voting_model.keras')

# Predict the probabilities
states_probabilities = fnn_model.predict(X_states_normalized)

# Convert probabilities to binary predictions
states_predictions = (states_probabilities > 0.5).astype(int).flatten()

# Calculate evaluation metrics
test_accuracy = accuracy_score(y_states, states_predictions)
test_precision = precision_score(y_states, states_predictions)
test_recall = recall_score(y_states, states_predictions)
f1 = f1_score(y_states, states_predictions)

# Print the evaluation metrics
print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Test Precision: {test_precision:.4f}")
print(f"Test Recall: {test_recall:.4f}")
print(f"F1 Score: {f1:.4f}")

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
Test Accuracy: 0.5882
Test Precision: 1.0000
Test Recall: 0.1923
F1 Score: 0.3226
